In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Hypothesis
$H(X)=\frac{1}{1+e^{XW}}$
### Cost
$cost(W)=-\frac{1}{m}\sum{y\log{H(X)}+(1-y)\log{(1-H(X))}}$

In [2]:
torch.manual_seed(1)

In [3]:
# data 
x_data = torch.tensor([[1, 2],
          [2, 3],
          [3, 1],
          [4, 3],
          [5, 3],
          [6, 2]],dtype=torch.float)
y_data = torch.tensor([[0],
          [0],
          [0],
          [1],
          [1],
          [1]], dtype=torch.float)
x_data, y_data

(tensor([[1., 2.],
         [2., 3.],
         [3., 1.],
         [4., 3.],
         [5., 3.],
         [6., 2.]]),
 tensor([[0.],
         [0.],
         [0.],
         [1.],
         [1.],
         [1.]]))

In [4]:
# parameters: weight and bias
W = torch.zeros(2,1)
b = torch.zeros(1)

In [5]:
# hypothesis: softmax(sigmoid)
hypothesis = 1 / (1 + torch.exp(-(x_data@W+b)))
hypothesis

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]])

In [6]:
# sigmoid function
hypothesis = torch.sigmoid(-(x_data@W+b))
hypothesis

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]])

In [7]:
# loss function: binary cross entropy
cost = -(y_data * torch.log(hypothesis) + (1-y_data)*torch.log(1-hypothesis))
cost

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]])

In [8]:
cost = -(y_data * torch.log(hypothesis) + (1-y_data)*torch.log(1-hypothesis)).mean()
cost

tensor(0.6931)

In [9]:
# binary cross entropy function
F.binary_cross_entropy(hypothesis, y_data)

tensor(0.6931)

In [10]:
nn.BCELoss()(hypothesis, y_data)

tensor(0.6931)

In [11]:
# training model
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_data.matmul(W) + b) # or .mm or @
    cost = -(y_data * torch.log(hypothesis) + 
             (1 - y_data) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [20]:
# check result
torch.sigmoid(x_data.matmul(W) + b)

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)

In [18]:
torch.where(torch.sigmoid(x_data.matmul(W) + b) > 0.5, 1, 0)

tensor([[0],
        [0],
        [0],
        [1],
        [1],
        [1]])

In [19]:
sum(torch.where(torch.sigmoid(x_data.matmul(W) + b) > 0.5, 1, 0) == y_data)

tensor([6])

In [21]:
# Example: diabetes dataset
from sklearn.datasets import load_diabetes
dataset = load_diabetes()
x_diabetes = torch.tensor(dataset.data, dtype=torch.float)
y_diabetes = torch.tensor(dataset.target, dtype=torch.float).view(-1,1)
y_diabetes = torch.where(y_diabetes > 180, 1., 0.)

In [22]:
x_diabetes.shape, y_diabetes.shape

(torch.Size([442, 10]), torch.Size([442, 1]))

In [23]:
# training model
W = torch.zeros((10, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

# training
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_diabetes @ W + b) # or .mm or @
    cost = nn.BCELoss()(hypothesis, y_diabetes)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.604198
Epoch  200/1000 Cost: 0.574378
Epoch  300/1000 Cost: 0.552757
Epoch  400/1000 Cost: 0.536597
Epoch  500/1000 Cost: 0.524155
Epoch  600/1000 Cost: 0.514313
Epoch  700/1000 Cost: 0.506335
Epoch  800/1000 Cost: 0.499732
Epoch  900/1000 Cost: 0.494167
Epoch 1000/1000 Cost: 0.489404


In [27]:
# check result
prediction = torch.tensor([[1.] if h >= torch.tensor([0.5]) else [0.] for h in hypothesis])
prediction[:5]

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [28]:
y_diabetes[:5]

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.]])

In [31]:
sum(prediction[:5] == y_diabetes[:5])/5

tensor([0.8000])

In [35]:
# binary classifier Class
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(10, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

    def fit(self, X, y, epochs=1000, lr=0.01, optimizer='SGD', loss='BCELoss', print_every=100):
        # optimizer
        optimizer = getattr(optim, optimizer)(self.net.parameters(), lr=lr)

        # loss function
        loss_fn = getattr(nn, loss)()

        # training
        for epoch in range(1, epochs+1):
            # forward
            hypothesis = self.net(X)
            cost = loss_fn(hypothesis, y)

            # backward
            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            # print
            if epoch % print_every == 0:
                print(f'Epoch: {epoch}, Loss: {cost.item():.5f}')

    def predict(self, x):
        return torch.where(self.net(x) >= 0.5, 1., 0.).view(-1, 1)

In [49]:
model = BinaryClassifier()
model

BinaryClassifier(
  (net): Sequential(
    (0): Linear(in_features=10, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [50]:
model.fit(x_diabetes, y_diabetes, lr=0.001, epochs=5000, optimizer='Adam', print_every=500)

Epoch: 500, Loss: 0.64882
Epoch: 1000, Loss: 0.60129
Epoch: 1500, Loss: 0.57702
Epoch: 2000, Loss: 0.56041
Epoch: 2500, Loss: 0.54690
Epoch: 3000, Loss: 0.53530
Epoch: 3500, Loss: 0.52505
Epoch: 4000, Loss: 0.51586
Epoch: 4500, Loss: 0.50757
Epoch: 5000, Loss: 0.50010


In [53]:
model.predict(x_diabetes[:5]) == y_diabetes[:5]

tensor([[ True],
        [ True],
        [ True],
        [False],
        [ True]])